In [1]:
import polars as pl
from dotenv import dotenv_values

# pl.Config.set_fmt_str_lengths(1000)
# pl.Config.set_fmt_table_cell_list_len(1000)
pl.Config.set_tbl_rows(1000)
uri = dotenv_values("../arkin/.env")['DATABASE_URL']

In [2]:
# Fetch all the features from the insights table
query = """SELECT DISTINCT feature_id FROM insights WHERE instrument_id = 'f5dd7db6-89da-4c68-b62e-6f80b763bef6'"""
df = pl.read_database_uri(query, uri, engine="connectorx")
print(df)

shape: (11, 1)
┌────────────────────┐
│ feature_id         │
│ ---                │
│ str                │
╞════════════════════╡
│ sma_60_vwap        │
│ vwap               │
│ hist_vol           │
│ spread_sma_volume  │
│ sma_5_vwap         │
│ cumulative_returns │
│ spread_sma_vwap    │
│ volume             │
│ sma_5_volume       │
│ sma_60_volume      │
│ returns            │
└────────────────────┘


In [3]:
# Fetching the feature data
query = """
SELECT
    event_time,
    MAX(CASE WHEN feature_id = 'vwap' THEN value END) AS vwap,
    MAX(CASE WHEN feature_id = 'hist_vol' THEN value END) AS hist_vol,
    MAX(CASE WHEN feature_id = 'sma_60_vwap' THEN value END) AS sma_60_vwap
    -- Add more features here if necessary
FROM 
    insights
WHERE 
    instrument_id = 'f5dd7db6-89da-4c68-b62e-6f80b763bef6'
GROUP BY 
    event_time
ORDER BY 
    event_time"""
df = pl.read_database_uri(query, uri, engine="connectorx")
df.head(10)

event_time,vwap,hist_vol,sma_60_vwap
"datetime[ns, UTC]",f64,f64,f64
2024-09-29 03:31:00 UTC,65789.721002,0.0,65789.721002
2024-09-29 03:32:00 UTC,65772.125229,0.000303,65780.923115
2024-09-29 03:33:00 UTC,65749.026696,0.000286,65770.290976
2024-09-29 03:34:00 UTC,65750.453704,0.000279,65765.331658
2024-09-29 03:35:00 UTC,65755.32467,0.000255,65763.33026
2024-09-29 03:36:00 UTC,65774.825567,0.00033,65765.246145
2024-09-29 03:37:00 UTC,65787.617991,0.000307,65768.442123
2024-09-29 03:38:00 UTC,65785.459727,0.000287,65770.569323
2024-09-29 03:39:00 UTC,65785.787167,0.000271,65772.260195


In [4]:
df.describe()

statistic,event_time,vwap,hist_vol,sma_60_vwap
str,str,f64,f64,f64
"""count""","""1860""",1860.0,1860.0,1860.0
"""null_count""","""0""",0.0,0.0,0.0
"""mean""","""2024-09-29 19:00:30+00:00""",65277.567421,0.000355,65313.120788
"""std""",null,625.678682,0.000173,582.315386
"""min""","""2024-09-29 03:31:00+00:00""",63253.755345,0.0,63626.00877
"""25%""","""2024-09-29 11:16:00+00:00""",64574.985027,0.000231,64608.901821
"""50%""","""2024-09-29 19:01:00+00:00""",65569.309704,0.000291,65583.421785
"""75%""","""2024-09-30 02:45:00+00:00""",65716.376318,0.000429,65723.633845
"""max""","""2024-09-30 10:30:00+00:00""",66014.038766,0.000981,65899.681839
